Наивный алгоритм

In [21]:
def native_search(text, pattern, debug = False):
    len_text, len_pattern = len(text), len(pattern)
    for i in range(len_text - len_pattern + 1):
        if text[i : i + len_pattern] == pattern:
            if debug: return None
            return i
    return None

In [22]:
def native_search_2(text, pattern, debug = False):
    counter = 0
    len_text, len_pattern = len(text), len(pattern)
    
    for i in range(len_text - len_pattern + 1):
        test = 0
        for j in range(len_pattern):
            counter += 1
            if text[i+j] == pattern[j]:
                test += 1
            else: break
            if test == len_pattern:
                if debug: return counter
                return i
    return None

Алгоритм Бойера-Мура-Хорспула


In [23]:
def boyer_moore_horspool_search(text, pattern, debug = False):
    counter = 0
    len_text, len_pattern = len(text), len(pattern)

    shift = dict()
    shift[pattern[-1]] = len_pattern
    for i in reversed(range(len(pattern) - 1 )):
        symbol = pattern[i]
        if symbol in shift:
            shift[symbol] = min(shift[symbol], len_pattern - i)
            continue
        shift[symbol] = len_pattern - i - 1
    i = 0
    while i < len_text:
        nextShift = 0
        match = True
        l = i + len_pattern - 1
        if l > len_text:
            return None
        for j in reversed(range(len_pattern)):
            counter += 1
            if pattern[j] == text[l]:
                nextShift = len_pattern
                l -= 1
            else:
                match = False
                if nextShift != 0:
                    i += nextShift
                    break
                if text[l] in shift:
                    i += shift[text[l]]
                    break
                else:
                    i += len_pattern - 1
                    break
        if match:
            if debug: return counter
            return i
    return None

Алгоритм Рабина-Карпа

In [32]:
import sympy.ntheory as nt

def rabin_karp_search_2(text, pattern, debug=False):
    counter = 0

    d = len(set(list(text)))
    q = nt.nextprime(len(set(list(text))))
    len_pattern = len(pattern) 
    len_text    = len(text)

    pattern_hash = 0
    text_hash    = 0
    h = 1

    for _ in range(len_pattern-1):
        h = (h*d) % q

    for i in range(len_pattern): 
        pattern_hash = (d * pattern_hash + ord(pattern[i])) % q
        text_hash    = (d * text_hash    + ord(text[i])   ) % q

    for i in range(len_text - len_pattern + 1):
        if pattern_hash == text_hash:
            test = 0
            for j in range(len_pattern):
                counter += 1
                if text[i+j] == pattern[j]:
                    test += 1
                else: break
                if test == len_pattern:
                    if debug: return counter
                    return i
        if i < len_text - len_pattern: 
            text_hash = (d*(text_hash-ord(text[i])*h) + ord(text[i+len_pattern])) % q 
            if text_hash < 0: 
                text_hash = text_hash + q
    return None

In [25]:
def rabin_karp_search(text, pattern, debug=False):
    counter = 0
    len_text, len_pattern = len(text), len(pattern)

    hpattern = hash(pattern)
    for i in range(len_text-len_pattern+1):
        hs = hash(text[i:i+len_pattern])
        if hs == hpattern:
            test = 0
            for j in range(len_pattern):
                counter += 1
                if text[i+j] == pattern[j]:
                    test += 1
                else: break
                if test == len_pattern:
                    if debug: return counter
                    return i
    return None

Алгоритм Кнутта-Мориса-Пратта


In [54]:
def get_prefix(_pattern):
    prefix_doct = {0: 0}
    for i in range(1, len(_pattern)):
        j = prefix_doct[i - 1]
        while j > 0 and _pattern[j] != _pattern[i]:
            j = prefix_doct[j - 1]
        if _pattern[j] == _pattern[i]:
            j += 1
        prefix_doct[i] = j
    return prefix_doct


def knuth_morris_pratt_search(text, pattern, debug = False):
    counter = 0
    ans = None

    len_text, len_pattern = len(text), len(pattern)
    prefix_doct = get_prefix(pattern)

    index_text = index_pattern = 0

    while index_text < len_text and index_pattern < len_pattern:
        counter += 1
        if pattern[index_pattern] == text[index_text]:
            index_text    += 1
            index_pattern += 1
        elif index_pattern == 0:
            index_text += 1
        else:
            index_pattern = prefix_doct.get(index_pattern - 1)
    else:
        if index_pattern == len_pattern:
            ans = index_text - index_pattern
    if debug: return counter
    return ans

In [27]:
def c_find(text, pattern, debug = False):
    if debug: return None
    return text.find(pattern)

# Test

In [8]:
!wget https://github.com/alexgiving/data_collab/raw/main/benchmarks.rar >> /dev/null
!unrar x benchmarks.rar  >> /dev/null

bad_t_1 = open(f"benchmarks/bad_t_1.txt", "r").read()
bad_t_2 = open(f"benchmarks/bad_t_2.txt", "r").read()
bad_t_3 = open(f"benchmarks/bad_t_3.txt", "r").read()
bad_t_4 = open(f"benchmarks/bad_t_4.txt", "r").read()

bad_w_1 = open(f"benchmarks/bad_w_1.txt", "r").read()
bad_w_2 = open(f"benchmarks/bad_w_2.txt", "r").read()
bad_w_3 = open(f"benchmarks/bad_w_3.txt", "r").read()
bad_w_4 = open(f"benchmarks/bad_w_4.txt", "r").read()

good_t_1 = open(f"benchmarks/good_t_1.txt", "r").read()
good_t_2 = open(f"benchmarks/good_t_2.txt", "r").read()
good_t_3 = open(f"benchmarks/good_t_3.txt", "r").read()
good_t_4 = open(f"benchmarks/good_t_4.txt", "r").read()

good_w_1 = open(f"benchmarks/good_w_1.txt", "r").read()
good_w_2 = open(f"benchmarks/good_w_2.txt", "r").read()
good_w_3 = open(f"benchmarks/good_w_3.txt", "r").read()
good_w_4 = open(f"benchmarks/good_w_4.txt", "r").read()

--2022-04-29 18:21:37--  https://github.com/alexgiving/data_collab/raw/main/benchmarks.rar
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexgiving/data_collab/main/benchmarks.rar [following]
--2022-04-29 18:21:37--  https://raw.githubusercontent.com/alexgiving/data_collab/main/benchmarks.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8285 (8.1K) [application/octet-stream]
Saving to: ‘benchmarks.rar’

benchmarks.rar      100%[===================>]   8.09K  --.-KB/s    in 0s      

2022-04-29 18:21:38 (56.4 MB/s) - ‘benchmarks.rar’ saved [8285/8285]



In [17]:
def timer(_func):
    start_timer = time.perf_counter()
    _func
    stop_timer = time.perf_counter()
    return stop_timer - start_timer

In [36]:
from statistics import mean 
import pandas as pd
import time

####################################
c_find_time = [] 

native_time               = []
native_time_2             = []

boyer_moore_horspool_time = []

knuth_morris_pratt_time   = []

rabin_karp_time           = []
rabin_karp_time_2         = []
####################################


iters = 10


test_cases = {"bad_1" : (bad_t_1,  bad_w_1),
              "bad_2" : (bad_t_2,  bad_w_2),
              "bad_3" : (bad_t_3,  bad_w_3),
              "bad_4" : (bad_t_4,  bad_w_4),
              "good_1": (good_t_1, good_w_1),
              "good_2": (good_t_2, good_w_2),
              "good_3": (good_t_3, good_w_3),
              "good_4": (good_t_4, good_w_4),
              "stress_test_1": (str("a" * 10000000 + "b"), "aaaaaaaaaaab")
}


search_funcs = {"Python-Search"               : (c_find,                      c_find_time),
                "Native-Search"               : (native_search,               native_time),
                "Native-Search_2"             : (native_search_2,             native_time_2),
                "Boyer–Moore-Horspool-Search" : (boyer_moore_horspool_search, boyer_moore_horspool_time),
                "Knuth-Morris-Pratt-Search"   : (knuth_morris_pratt_search,   knuth_morris_pratt_time),
                "Rabin-Karp-Search"           : (rabin_karp_search,           rabin_karp_time),
                "Rabin-Karp-Search_2"         : (rabin_karp_search_2,         rabin_karp_time_2)
}


for test_case in test_cases:
    text, word = test_cases.get(test_case)
    for func_name in search_funcs:
        func, time_array = search_funcs.get(func_name)
        if func(text, word) == text.find(word):
            temp_array = [timer(func(text, word)) for _ in range(iters)]
            time_array.append( f'sec={"{:.2e}".format(mean(temp_array))}, iter={func(text, word, True)}')
        else: time_array.append('ERROR')

data = pd.DataFrame(data=[i[1:][0] for i in search_funcs.values()],
            columns=test_cases.keys(),
            index=search_funcs.keys()
            ).T


display(data)

,Python-Search,Native-Search,Native-Search_2,Boyer–Moore-Horspool-Search,Knuth-Morris-Pratt-Search,Rabin-Karp-Search,Rabin-Karp-Search_2
bad_1,"sec=1.57e-07, iter=None","sec=1.36e-07, iter=9","sec=1.99e-07, iter=18","sec=1.37e-07, iter=10","sec=1.35e-07, iter=19","sec=1.35e-07, iter=11","sec=1.36e-07, iter=2"
bad_2,"sec=1.34e-07, iter=None","sec=1.39e-07, iter=91","sec=1.37e-07, iter=910","sec=1.61e-07, iter=100","sec=1.38e-07, iter=191","sec=1.37e-07, iter=101","sec=1.56e-07, iter=10"
bad_3,"sec=1.37e-07, iter=None","sec=1.58e-07, iter=901","sec=3.54e-07, iter=90100","sec=1.88e-07, iter=1000","sec=1.57e-07, iter=1901","sec=1.65e-07, iter=1001","sec=1.70e-07, iter=100"
bad_4,"sec=1.61e-07, iter=None","sec=1.73e-07, iter=4001","sec=6.36e-07, iter=4001000","sec=2.18e-07, iter=5000","sec=2.03e-07, iter=9001","sec=2.28e-07, iter=5001","sec=4.47e-07, iter=1000"
good_1,"sec=1.37e-07, iter=None","sec=1.53e-07, iter=600","sec=1.51e-07, iter=633","sec=1.59e-07, iter=84","sec=1.67e-07, iter=634","sec=1.55e-07, iter=617","sec=1.72e-07, iter=32"
good_2,"sec=1.40e-07, iter=None","sec=1.60e-07, iter=611","sec=1.79e-07, iter=695","sec=1.55e-07, iter=105","sec=1.85e-07, iter=696","sec=2.71e-07, iter=696","sec=2.18e-07, iter=91"
good_3,"sec=1.43e-07, iter=None","sec=1.65e-07, iter=1630","sec=1.68e-07, iter=2074","sec=1.76e-07, iter=427","sec=1.74e-07, iter=2067","sec=1.79e-07, iter=2011","sec=2.17e-07, iter=410"
good_4,"sec=1.41e-07, iter=None","sec=2.38e-07, iter=9523","sec=3.59e-07, iter=9614","sec=2.36e-07, iter=425","sec=3.18e-07, iter=9615","sec=2.91e-07, iter=9615","sec=4.62e-07, iter=240"
stress_test_1,"sec=6.06e-07, iter=None","sec=6.15e-07, iter=9999990","sec=4.73e-07, iter=119999880","sec=6.56e-07, iter=10000001","sec=5.77e-07, iter=19999991","sec=4.64e-07, iter=10000002","sec=4.93e-07, iter=12"


# Results


Notes:

*   Разница между скоростями `Native Search` заключается в оптимизации интерпретатором сравнения слов.

*   Разница между скоростями `Rabin-Karp-Search` заключается в оптимизации интерпретатором функции хеширования.

---

Фактически нас интересует сравнение наших функций поиска, а не оптимизации вычислений интерпретатора.
Поэтому основными функциями для сравнения будут `Native Search_2`, `Rabin-Karp-Search_2`, `Knuth-Morris-Pratt-Search`, `Boyer–Moore-Horspool-Search`


Итоговая таблица:







In [43]:
display(data.drop(['Python-Search', 'Native-Search', 'Rabin-Karp-Search'], axis=1))

,Native-Search_2,Boyer–Moore-Horspool-Search,Knuth-Morris-Pratt-Search,Rabin-Karp-Search_2
bad_1,"sec=1.99e-07, iter=18","sec=1.37e-07, iter=10","sec=1.35e-07, iter=19","sec=1.36e-07, iter=2"
bad_2,"sec=1.37e-07, iter=910","sec=1.61e-07, iter=100","sec=1.38e-07, iter=191","sec=1.56e-07, iter=10"
bad_3,"sec=3.54e-07, iter=90100","sec=1.88e-07, iter=1000","sec=1.57e-07, iter=1901","sec=1.70e-07, iter=100"
bad_4,"sec=6.36e-07, iter=4001000","sec=2.18e-07, iter=5000","sec=2.03e-07, iter=9001","sec=4.47e-07, iter=1000"
good_1,"sec=1.51e-07, iter=633","sec=1.59e-07, iter=84","sec=1.67e-07, iter=634","sec=1.72e-07, iter=32"
good_2,"sec=1.79e-07, iter=695","sec=1.55e-07, iter=105","sec=1.85e-07, iter=696","sec=2.18e-07, iter=91"
good_3,"sec=1.68e-07, iter=2074","sec=1.76e-07, iter=427","sec=1.74e-07, iter=2067","sec=2.17e-07, iter=410"
good_4,"sec=3.59e-07, iter=9614","sec=2.36e-07, iter=425","sec=3.18e-07, iter=9615","sec=4.62e-07, iter=240"
stress_test_1,"sec=4.73e-07, iter=119999880","sec=6.56e-07, iter=10000001","sec=5.77e-07, iter=19999991","sec=4.93e-07, iter=12"
